<a href="https://colab.research.google.com/github/ashleyak7/MSCI-151_ashley/blob/main/CW5_Koesnadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pulp as pl
from collections import defaultdict
import os
import csv

In [2]:
baristas=["Max","Jiwa", "Fore","Donna","Paul"]
days=list(range(1,7))
blocks=list(range(1,5))
H=4
cost={"Max":50000,"Jiwa":50000,"Fore":50000,"Donna":150000,"Paul":150000}
etype={"Max":"P","Jiwa":"P","Fore":"P","Donna":"F","Paul":"F"}
minWeekly_default={"Max":12,"Jiwa":12,"Fore":12, "Donna":36, "Paul":36}

In [3]:
avail = {
    ("Max", 1): 8, ("Max", 2): 8, ("Max", 3): 8, ("Max", 4): 0, ("Max", 5): 8, ("Max", 6): 4, ("Max", 7): 4,
    ("Jiwa", 1): 4, ("Jiwa", 2): 4, ("Jiwa", 3): 8, ("Jiwa", 4): 8, ("Jiwa", 5): 0, ("Jiwa", 6): 4, ("Jiwa", 7): 0,
    ("Fore", 1): 8, ("Fore", 2): 0, ("Fore", 3): 8, ("Fore", 4): 8, ("Fore", 5): 8, ("Fore", 6): 8, ("Fore", 7): 0,
    ("Donna", 1): 12, ("Donna", 2): 12, ("Donna", 3): 12, ("Donna", 4): 12, ("Donna", 5): 12, ("Donna", 6): 12, ("Donna", 7): 8,
    ("Paul", 1): 12, ("Paul", 2): 8, ("Paul", 3): 12, ("Paul", 4): 12, ("Paul", 5): 8, ("Paul", 6): 12, ("Paul", 7): 12
}

In [4]:
days = {
    1: "Monday",
    2: "Tuesday",
    3: "Wednesday",
    4: "Thursday",
    5: "Friday",
    6: "Saturday",
    7: "Sunday"
}

In [5]:
Block_Name={1: "07:00-11:00", 2: "11:00-15:00", 3: "15:00-19:00", 4: "19:00-23:00"}

In [ ]:
# Create the problem
prob = pl.LpProblem("Barista_Scheduling", pl.LpMinimize)

# Define the set of days to iterate over (1 to 7)
days_indices = list(days.keys()) # days is now a dict {1:'Monday', ...}

# Decision Variables
# x[(barista, day_index, block)] is 1 if barista works block k on day d, 0 otherwise
x = pl.LpVariable.dicts("Shift",
                        ((b, d_idx, k) for b in baristas for d_idx in days_indices for k in blocks),
                        0, 1, pl.LpBinary)

# Objective Function: Minimize total hours worked
prob += pl.lpSum(x[(b, d_idx, k)] * H
                 for b in baristas for d_idx in days_indices for k in blocks), "Total Hours Worked"

# Constraints

# 1. Each block on each day must be covered by at least one barista
for d_idx in days_indices:
    for k in blocks:
        prob += pl.lpSum(x[(b, d_idx, k)] for b in baristas) >= 1, f"Cover_Block_{d_idx}_{k}"

# 2. Barista daily availability constraint: total hours worked by a barista on a day cannot exceed their available hours
for b in baristas:
    for d_idx in days_indices:
        prob += pl.lpSum(x[(b, d_idx, k)] * H for k in blocks) <= avail[(b, d_idx)], f"Daily_Availability_{b}_{d_idx}"

# 3. Minimum Weekly Hours for each barista
for b in baristas:
    prob += pl.lpSum(x[(b, d_idx, k)] * H
                     for d_idx in days_indices for k in blocks) >= minWeekly_default[b], f"Min_Weekly_Hours_{b}"

# Solve the problem
prob.solve()

# Print the solution status
print("Status:", pl.LpStatus[prob.status])

# Print the schedule
if prob.status == 1: # Changed from pl.LpStatus.Optimal to 1 to avoid AttributeError
    print("\nOptimal Schedule:")
    schedule = defaultdict(lambda: defaultdict(list))
    for v in prob.variables():
        if v.varValue > 0.5: # If the variable is active
            if v.name.startswith("Shift_"):
                # Extract barista, day, block from variable name
                # Variable names are like Shift_('Max',_1,_1) or similar due to tuple indexing
                parts = v.name.split('_')
                # Reconstruct parts to handle pulp's string representation of tuples
                barista_name = parts[1].strip("('").strip("',")
                day_index = int(parts[2].strip(","))
                block_index = int(parts[3].strip(")"))
                schedule[barista_name][day_index].append(block_index)

    # Pretty print the schedule
    for b in baristas:
        print(f"\nBarista: {b}")
        total_weekly_hours = 0
        for d_idx in days_indices:
            daily_hours = 0
            if schedule[b][d_idx]:
                blocks_worked = sorted(schedule[b][d_idx])
                blocks_str = ", ".join([Block_Name[k] for k in blocks_worked])
                daily_hours = len(blocks_worked) * H
                print(f"  {days[d_idx]}: {blocks_str} ({daily_hours} hours)")
            else:
                print(f"  {days[d_idx]}: No shifts")
            total_weekly_hours += daily_hours
        print(f"  Total Weekly Hours: {total_weekly_hours} (Min: {minWeekly_default[b]})\n")

    print(f"Total Hours Worked (Objective Value): {pl.value(prob.objective)} hours")

else:
    print("No optimal solution found.")

Status: Optimal

Optimal Schedule:

Barista: Max
  Monday: No shifts
  Tuesday: 19:00-23:00 (4 hours)
  Wednesday: 11:00-15:00 (4 hours)
  Thursday: No shifts
  Friday: No shifts
  Saturday: 15:00-19:00 (4 hours)
  Sunday: No shifts
  Total Weekly Hours: 12 (Min: 12)


Barista: Jiwa
  Monday: 19:00-23:00 (4 hours)
  Tuesday: 15:00-19:00 (4 hours)
  Wednesday: 07:00-11:00 (4 hours)
  Thursday: No shifts
  Friday: No shifts
  Saturday: 11:00-15:00 (4 hours)
  Sunday: No shifts
  Total Weekly Hours: 16 (Min: 12)


Barista: Fore
  Monday: No shifts
  Tuesday: No shifts
  Wednesday: 15:00-19:00 (4 hours)
  Thursday: 15:00-19:00 (4 hours)
  Friday: 11:00-15:00 (4 hours)
  Saturday: No shifts
  Sunday: No shifts
  Total Weekly Hours: 12 (Min: 12)


Barista: Donna
  Monday: 11:00-15:00 (4 hours)
  Tuesday: 07:00-11:00 (4 hours)
  Wednesday: No shifts
  Thursday: 07:00-11:00, 11:00-15:00 (8 hours)
  Friday: 07:00-11:00 (4 hours)
  Saturday: 07:00-11:00, 19:00-23:00 (8 hours)
  Sunday: 11:00-15:

In [7]:
# Fairness model builder
# ----------------------------
def build_fairness_model(baristas, days, blocks, H, cost, avail, etype, minWeekly, baseline_cost, allow_blocks=None, name="Fairness"):
    """
    Build fairness MILP:
    - minimize Hmax - Hmin
    - subject to coverage, availability, contractual caps, weekly minima
    - plus budget cap: total_cost <= 1.02 * baseline_cost
    Uses flat y[(o,d,b)] variables.
    """
    model = pl.LpProblem(name, pl.LpMinimize)

    if allow_blocks is None:
        allow_blocks = {d: set(blocks) for d in days}
    else:
        allow_blocks = {d: set(bs) for d, bs in allow_blocks.items()}

    # Binary assignment vars
    y = pl.LpVariable.dicts("y", [(o, d, b) for o in baristas for d in days for b in allow_blocks[d]], lowBound=0, upBound=1, cat="Binary")

    # Continuous hours vars and extrema
    hours_o = pl.LpVariable.dicts("hours", baristas, lowBound=0, cat="Continuous")
    Hmax = pl.LpVariable("Hmax", lowBound=0, cat="Continuous")
    Hmin = pl.LpVariable("Hmin", lowBound=0, cat="Continuous")

    # Objective: minimize gap
    model += Hmax - Hmin, "FairnessGap"

    # Coverage
    for d in days:
        for b in allow_blocks[d]:
            model += pl.lpSum(y[(o, d, b)] for o in baristas) >= 1, f"cover_d{d}_b{b}"

    # Daily availability and block caps
    for o in baristas:
        for d in days:
            model += H * pl.lpSum(y[(o, d, b)] for b in allow_blocks[d]) <= avail[(o, d)], f"avail_{o}_{d}"
            cap = 2 if etype[o] == "P" else 3
            model += pl.lpSum(y[(o, d, b)] for b in allow_blocks[d]) <= cap, f"blockCap_{o}_{d}"

    # Weekly minima
    for o in baristas:
        model += H * pl.lpSum(y[(o, d, b)] for d in days for b in allow_blocks[d]) >= minWeekly[o], f"weeklyMin_{o}"

    # Link hours_o and bound to Hmax/Hmin
    for o in baristas:
        model += hours_o[o] == H * pl.lpSum(y[(o, d, b)] for d in days for b in allow_blocks[d]), f"hours_link_{o}"
        model += hours_o[o] <= Hmax, f"leq_Hmax_{o}"
        model += hours_o[o] >= Hmin, f"geq_Hmin_{o}"

    # Budget cap (≤ 1.02 * baseline_cost)
    total_cost = pl.lpSum(H * cost[o] * y[(o, d, b)] for o in baristas for d in days for b in allow_blocks[d])
    model += total_cost <= 1.02 * baseline_cost, "budget_cap"

    return model, y, allow_blocks


In [8]:
# Scenario
# ----------------------------
def solve_baseline():
    model, y, allow_blocks = build_staffing_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_default, name="Baseline"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)  # 5 min
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)


def solve_close_at_19():
    allow_blocks = {d: {1, 2, 3} for d in days}
    model, y, allow_blocks = build_staffing_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_default, allow_blocks=allow_blocks, name="CloseAt19"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)


def solve_pt_min16():
    minWeekly_pt16 = {o: (16 if etype[o] == "P" else 36) for o in baristas}
    model, y, allow_blocks = build_staffing_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_pt16, name="PTMin16"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)


def solve_fairness(baseline_cost):
    model, y, allow_blocks = build_fairness_model(
        baristas, days, blocks, H, cost, avail, etype, minWeekly_default, baseline_cost, name="Fairness"
    )
    solver = pl.PULP_CBC_CMD(msg=True, timeLimit=300)
    model.solve(solver)
    return model, extract_results(model, y, baristas, days, blocks, allow_blocks, H)

In [9]:
# Main execution
# ----------------------------
def main():
    print("CW5 MILP models using PuLP (CBC).")

    # Baseline
    baseline_model, (status_base, base_cost, base_hours, base_sched) = solve_baseline()
    print_summary("BASELINE (open 07:00–23:00)", status_base, base_cost, base_hours, base_sched)
    save_schedule_csv("results/baseline_schedule.csv", base_hours, base_sched, days, blocks)

    # Scenario A: close at 19:00 (remove block 4)
    sA_model, (status_sA, sA_cost, sA_hours, sA_sched) = solve_close_at_19()
    print_summary("SCENARIO A — CLOSE AT 19:00 (remove block 4)", status_sA, sA_cost, sA_hours, sA_sched)
    save_schedule_csv("results/close19_schedule.csv", sA_hours, sA_sched, days, [1, 2, 3])  # only 3 blocks

    # Scenario B: part-time weekly minimum = 16 hours
    sB_model, (status_sB, sB_cost, sB_hours, sB_sched) = solve_pt_min16()
    print_summary("SCENARIO B — PART-TIME MIN = 16 HOURS", status_sB, sB_cost, sB_hours, sB_sched)
    save_schedule_csv("results/ptmin16_schedule.csv", sB_hours, sB_sched, days, blocks)

    # Fairness model (≤ 102% of baseline cost)
    fairness_model, (status_fair, fair_cost, fair_hours, fair_sched) = solve_fairness(base_cost)
    print_summary("FAIRNESS VARIANT (minimize Hmax-Hmin; cost ≤ 102% of baseline)", status_fair, fair_cost, fair_hours, fair_sched)
    save_schedule_csv("results/fairness_schedule.csv", fair_hours, fair_sched, days, blocks)

    # Quick cost summary printed
    print("--- COST SUMMARY (IDR) ---")
    print(f"Baseline:  {int(base_cost):,} (status: {status_base})")
    print(f"Close@19:  {int(sA_cost):,} (status: {status_sA})")
    print(f"PT min16:  {int(sB_cost):,} (status: {status_sB})") # Corrected typo here
    print(f"Fairness:  {int(fair_cost):,} (status: {status_fair})")
    print("--------------------------\n")

    # remind user of CW5 pdf path for report
    print("Coursework PDF (for your report / README):") # Changed this to remove CW5_PDF_PATH


if __name__ == "__main__":
    # CW5_PDF_PATH is not defined globally, so remove it or define it as a placeholder
    # For this example, I'll remove the variable from the print statement.
    main()


CW5 MILP models using PuLP (CBC).

BASELINE (open 07:00–23:00)
Solver status: Optimal
Total weekly cost (IDR): 12,800,000

Weekly hours per barista:
  - Max   : 12 hours
  - Jiwa  : 12 hours
  - Fore  : 16 hours
  - Donna : 36 hours
  - Paul  : 36 hours

Daily × Block schedule:
  Monday:
    [07:00-11:00] → Donna
    [11:00-15:00] → Paul
    [15:00-19:00] → Max
    [19:00-23:00] → Jiwa
  Tuesday:
    [07:00-11:00] → Donna
    [11:00-15:00] → Donna
    [15:00-19:00] → Jiwa
    [19:00-23:00] → Max
  Wednesday:
    [07:00-11:00] → Paul
    [11:00-15:00] → Fore
    [15:00-19:00] → Fore
    [19:00-23:00] → Paul
  Thursday:
    [07:00-11:00] → Donna
    [11:00-15:00] → Paul
    [15:00-19:00] → Donna
    [19:00-23:00] → Paul
  Friday:
    [07:00-11:00] → Paul
    [11:00-15:00] → Fore
    [15:00-19:00] → Fore
    [19:00-23:00] → Paul
  Saturday:
    [07:00-11:00] → Donna
    [11:00-15:00] → Jiwa
    [15:00-19:00] → Donna
    [19:00-23:00] → Max
  Sunday:
    [07:00-11:00] → Paul
    [11:00-15: